In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip install emoji --upgrade
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*
    !python -m spacy download en_core_web_lg
    !pip install pyldavis
    !pip install gensim
    !pip install chart_studio
    !pip install --upgrade autopep8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=5dc2b1011b9feeee02dac8c1b1db46d996ab8660cfd78e64e81e9e3445099d8a
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 245 kB 5.1 MB/s 
     |████████████████████████████████| 4.7 MB 47.8 MB/s 
     |████████████████████████████████| 102 kB 48.3 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 690 kB 48.9 MB/s 
     |████████████████████████████████| 355 kB 13.0 MB/s 
     |████████████████████████████████| 812 kB 45.9 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none

In [3]:
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
from collections import Counter

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

#Natural Language Processing (NLP)

import gensim

from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint


/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df_2015 = pd.read_excel('drive/MyDrive/AI ML lab project data/2015_clean_data.xlsx')
df_2019 = pd.read_excel('drive/MyDrive/AI ML lab project data/2019_clean_data.xlsx')
df_2022 = pd.read_excel('drive/MyDrive/AI ML lab project data/2022_clean_data_train.xlsx')

In [7]:
df_main = pd.concat([df_2015,df_2019,df_2022])

In [8]:
df_main

,clean_text,Party,Textblob_Sentiments,Emotions
0,living dream. cameraman camera cameraception c...,ndp,0.000000,Neutral
1,justin trudeau's reasons thanksgiving. today's...,liberals,0.000000,Neutral
2,butt......butt......we're allergic latex! make...,unrelated tweets,0.000000,Neutral
3,massive explosions peace march turkey. killed....,unrelated tweets,-0.100000,Negative
4,mulcair suggests bad blood trudeau worldwidehe...,ndp,-0.700000,Negative
...,...,...,...,...
4995,congress for progressive change and the all ni...,unrelated tweets,0.318182,Positive
4996,remember when they lost a write in campaign in...,other_election_topics,0.136364,Positive
4997,liz is going to change party affiliation. her ...,unrelated tweets,0.069024,Positive
4998,what is all this about coming back to democrat...,unrelated tweets,0.068182,Positive


In [9]:
import nltk as nt
from nltk.corpus import stopwords
nt.download('stopwords')
sw = stopwords.words( 'english' )
my_list = ['I','The',"b'It",'On']
sw = sw + my_list
def remove_stopwords(text):
    l = []
    for i in str(text).split():
        if i not in sw:
            l.append(i.strip('\'\"'))
    return l

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
df_main['tokens'] = df_main['clean_text'].apply(remove_stopwords)


In [11]:
df_main

,clean_text,Party,Textblob_Sentiments,Emotions,tokens
0,living dream. cameraman camera cameraception c...,ndp,0.000000,Neutral,"[living, dream., cameraman, camera, cameracept..."
1,justin trudeau's reasons thanksgiving. today's...,liberals,0.000000,Neutral,"[justin, trudeau's, reasons, thanksgiving., to..."
2,butt......butt......we're allergic latex! make...,unrelated tweets,0.000000,Neutral,"[butt......butt......we're, allergic, latex!, ..."
3,massive explosions peace march turkey. killed....,unrelated tweets,-0.100000,Negative,"[massive, explosions, peace, march, turkey., k..."
4,mulcair suggests bad blood trudeau worldwidehe...,ndp,-0.700000,Negative,"[mulcair, suggests, bad, blood, trudeau, world..."
...,...,...,...,...,...
4995,congress for progressive change and the all ni...,unrelated tweets,0.318182,Positive,"[congress, progressive, change, nigerian, peop..."
4996,remember when they lost a write in campaign in...,other_election_topics,0.136364,Positive,"[remember, lost, write, campaign, buffalo?, lo..."
4997,liz is going to change party affiliation. her ...,unrelated tweets,0.069024,Positive,"[liz, going, change, party, affiliation., fath..."
4998,what is all this about coming back to democrat...,unrelated tweets,0.068182,Positive,"[coming, back, democratic, politics, mass, mur..."


In [12]:
df_main['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df_main['tokens']]

In [13]:
df_main

,clean_text,Party,Textblob_Sentiments,Emotions,tokens,tokens_back_to_text
0,living dream. cameraman camera cameraception c...,ndp,0.000000,Neutral,"[living, dream., cameraman, camera, cameracept...",living dream. cameraman camera cameraception c...
1,justin trudeau's reasons thanksgiving. today's...,liberals,0.000000,Neutral,"[justin, trudeau's, reasons, thanksgiving., to...",justin trudeau's reasons thanksgiving. today's...
2,butt......butt......we're allergic latex! make...,unrelated tweets,0.000000,Neutral,"[butt......butt......we're, allergic, latex!, ...",butt......butt......we're allergic latex! make...
3,massive explosions peace march turkey. killed....,unrelated tweets,-0.100000,Negative,"[massive, explosions, peace, march, turkey., k...",massive explosions peace march turkey. killed....
4,mulcair suggests bad blood trudeau worldwidehe...,ndp,-0.700000,Negative,"[mulcair, suggests, bad, blood, trudeau, world...",mulcair suggests bad blood trudeau worldwidehe...
...,...,...,...,...,...,...
4995,congress for progressive change and the all ni...,unrelated tweets,0.318182,Positive,"[congress, progressive, change, nigerian, peop...",congress progressive change nigerian peoples p...
4996,remember when they lost a write in campaign in...,other_election_topics,0.136364,Positive,"[remember, lost, write, campaign, buffalo?, lo...",remember lost write campaign buffalo? lost eve...
4997,liz is going to change party affiliation. her ...,unrelated tweets,0.069024,Positive,"[liz, going, change, party, affiliation., fath...",liz going change party affiliation. father goi...
4998,what is all this about coming back to democrat...,unrelated tweets,0.068182,Positive,"[coming, back, democratic, politics, mass, mur...",coming back democratic politics mass murderer ...


In [17]:
id2word = Dictionary(df_main['tokens'])
print(len(id2word))

28229


In [18]:
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))

12414


In [19]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in df_main['tokens']]

In [20]:
# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [28]:
a = "hmmm steven del duca? if you are voting liberal you should seek professional help"
b = id2word.doc2bow(remove_stopwords(a))

In [31]:
b

[(53, 1),
 (156, 1),
 (511, 1),
 (561, 1),
 (6773, 1),
 (7821, 1),
 (7878, 1),
 (9662, 1)]

In [39]:
import pickle as plk

f = open("topic_model.pkl","wb")

plk.dump(base_model,f)

f.close()

In [40]:
topic_list = base_model[b]

In [55]:
n = []
for i in topic_list:
  n.append(i[1])

n = np.array(n)
b = n.argmax()
print(topic_list[b][0])

4


In [81]:
l = base_model.print_topic(4)

In [82]:
l = l.split('+')
for i in l:
  index = i.find('*')
  print(i[index+1:])

"canada" 
"ndp" 
"vote" 
"elections" 
"liberals" 
"conservative" 
"party" 
"election" 
"..." 
"like"


In [56]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

In [57]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [58]:
topics

['#elxn43 party #cdnpoli canada vote liberal elections new would democratic',
 'cdnpoli elxn ... #elxn43 #cdnpoli canada. trudeau canada elections r',
 'canada elections party new democratic conservative liberal canada. one ontario',
 'ndp #elxn43 liberals #cdnpoli party canada retweeted elections need vote',
 'canada ndp vote elections liberals conservative party election ... like']

In [ ]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
#elxn43 canada elections added retweeted #cdnpoli vote get us conservative

------ Topic 1 ------
party new democratic canada liberal canada. #elxn43 #cdnpoli elections party.

------ Topic 2 ------
#elxn43 #cdnpoli ndp liberals cdnpoli vote elections like get one

------ Topic 3 ------
party ... elxn canada cdnpoli elections vote ontario liberal conservative

------ Topic 4 ------
canada elections ndp liberals party #elxn43 election vote conservative #cdnpoli



In [ ]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=df_main['tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -8.329633107004549

Coherence Score:  0.3452651938577983


In [ ]:
import pyLDAvis.gensim

In [ ]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(base_model, corpus, id2word)

C:\Conda\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.017105 -0.005128       1        1  29.325059
2      0.021732 -0.022714       2        1  22.049674
1     -0.065168  0.099280       3        1  20.766655
3     -0.061255 -0.085095       4        1  15.745553
0      0.121796  0.013657       5        1  12.113060, topic_info=            Term         Freq        Total Category  logprob  loglift
147          new   755.000000   755.000000  Default  30.0000  30.0000
714   democratic   604.000000   604.000000  Default  29.0000  29.0000
19           ...   565.000000   565.000000  Default  28.0000  28.0000
391        party  1751.000000  1751.000000  Default  27.0000  27.0000
6           elxn   673.000000   673.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
226     election    45.873660   555.875274   Topic5  -5.8587  -0.3838
1574     ontario    43.507130   474.204535   Topic5  -5.9117  -0.2778
31           ndp    44.965339  1191.813821   Topic5  -5.8787  -1.1665
737        going    35.721674   242.164156   Topic5  -6.1088   0.1970
362         many    34.816537   189.813600   Topic5  -6.1345   0.4149

[421 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
3978       1  0.055869  #AndrewScheer
3978       2  0.726299  #AndrewScheer
3978       3  0.111738  #AndrewScheer
3978       4  0.111738  #AndrewScheer
3979       1  0.132877        #Canada
...      ...       ...            ...
8763       1  0.065655           🇺🇸🇨🇦
8763       4  0.065655           🇺🇸🇨🇦
8763       5  0.853511           🇺🇸🇨🇦
10501      2  0.121938              💡
10501      4  0.853565              💡

[937 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 4, 1])